# Bu kod ile TSEB için gerekli olan inputlar hazırlanmaktadır.

# Programın Kullanımı

1- daha önceden oluşturulmuş grid seçilir (fc ve Wc-Hc hesabı için gereklidir).

2- sırasıyla aynı çözünürlükteki hc(bitki boyu, DEM değil), yüksek çöz ndvi(fc sınıflandırması için), düş çöz ndvi (lai hesabı için, sorgumda)

3- toprak bitki ayrıştırmasında her tarih için eşik değerleri histogram ile güncellenmelidir.

4- geri kalanı kendi oluşturuyor, jupyter notebook un olduğu dizine kayıt yapıyor. ## şu kısmı güncelle

In [1]:
import geopandas as gpd
import rasterio
import rasterstats
import pandas as pd
import numpy as np
from geocube.api.core import make_geocube
import glob,os
from tkinter import filedialog
from tkinter import *
from datetime import datetime
from tqdm import tqdm

# Görüntülerin seçiminde burasıda kullanılabilir

In [2]:
klasor = r"J:\2021_TUBITAK_sorgum\2020sorgum\Altum_2020_paket"
dem = r"J:\2021_TUBITAK_sorgum\2020v2\ham\saved\VI\DEM"
path_hc= []
path_ndvi_high= []
path_ndvi_low= []


for root, dirname, files in os.walk(klasor):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-13:]=='ndvi_02cm.tif':
            path_ndvi_high.append(isim)

for root, dirname, files in os.walk(klasor):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-13:]=='ndvi_10cm.tif':
            path_ndvi_low.append(isim)
            
for root, dirname, files in os.walk(dem):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-8:]=='_DEM.tif':
            path_hc.append(isim)

print(len(path_ndvi_high))
print(len((path_ndvi_low)))
print(len(path_hc))



37
37
36


In [4]:
shp1 = r"J:\2021_TUBITAK_sorgum\2020sorgum\Altum_2020_paket\shp\2020_sorgum_FullArea_grid_10cm.shp"
shp= gpd.read_file(shp1)
print("Shp file dosyası Okundu")
    

for i in tqdm(range(1,len(path_ndvi_high)), colour="#00ff00"):

    ndvi_high_res = rasterio.open(path_ndvi_high[i], mode="r")
    ndvi_low_res  = rasterio.open(path_ndvi_low[i], mode="r")
    hc            = rasterio.open(path_hc[i-1], mode="r")
    
    _,isim=os.path.split(path_ndvi_high[i])
    isim= isim[:8]    
    
    
    kayitYeri=r"D:\pytseb\images\2020\\"+ isim
    
    # Klasör var mı yok mu onu kontrol ediyor.
    isExist = os.path.exists(kayitYeri)
    if not isExist:
        os.makedirs(kayitYeri)
        print(isim+" Klasörü Oluşturuldu!")
    

    
    
    ############  LAI Hesaplaması  ############
    LAI = ((ndvi_low_res.read(1) * hc.read(1)) * 6.0825 + 1.19134)
    LAI = np.where(LAI>0,LAI,0)

    kwargs = hc.meta
    kwargs.update(dtype=rasterio.float32, count=1)

    with rasterio.open(kayitYeri+'/'+(isim)+'_LAI.tif', 'w', **kwargs) as dst:
        dst.write_band(1, LAI.astype(rasterio.float32))
    print("{}_LAI Hesaplandı".format(isim))
    
    ############  Toprak-Bitki Ayrıştırma  ############
    
    top_bit = np.where(ndvi_high_res.read(1)>0.6,1,0) #her tarih için histogramdan düzenlenmelidir
    affine = ndvi_high_res.transform
    print("toprak bitki ayrıştırması tamamlandı")
     
    ###########  calculate zonal statistic  ############
    
    sum_top_bit = rasterstats.zonal_stats(shp, top_bit, affine= affine, stats=['sum'], 
                                     geojson_out = True)
    print("Zonal istatistik tamamlandı")
    
    ###########  Get sum of vegetation pixel values within the shp domain  ############
    
    sum_top_bit2 = []

    i=0

    while i<len(sum_top_bit):
        sum_top_bit2.append(sum_top_bit[i]["properties"])
        i=i+1
    print("Vej Pixel sayıları Toplandı")
    ###########  List to dataframe  ############
    
    sum_top_bit_df = pd.DataFrame(sum_top_bit2)
    
    "Listeler DataFrame'e Çevrildi"
        
    ###########  Calculation of Fc and Wc_Hc   ############
    
    sum_top_bit_df ["fc"] = (sum_top_bit_df["sum"]*ndvi_high_res.transform[0]*-ndvi_high_res.transform[4]) / (-hc.transform[4]*-hc.transform[4])   #0,02-> orj ndvi resolution, 0.1->shp domain resolution.
    sum_top_bit_df ["wc"] = (sum_top_bit_df["fc"] * (-hc.transform[4]))
    sum_top_bit_df ["geometry"] = shp.geometry
    
    
    "Fc ve Wc Hesaplamaları Gerçekleştirildi"
    ###########  DF to Shp File  ############
    df = gpd.GeoDataFrame(sum_top_bit_df, geometry='geometry')
    df.to_file(kayitYeri+'/'+'%s_grid.shp' %(isim), driver='ESRI Shapefile')  
    
    "Df To shp File Tamam"
  
    
    ###########  Shp to raster  ############
    input_geopackage= gpd.read_file(kayitYeri+'/'+'%s_grid.shp' %(isim))
    

    out_grid = make_geocube(
        vector_data=input_geopackage,
        resolution=(hc.transform[4], hc.transform[0]),
    )

    out_grid["fc"].rio.to_raster(kayitYeri+'/'+"%s_Fc.tif" %(isim))
    out_grid["wc"].rio.to_raster(kayitYeri+'/'+"%s_Wc.tif" %(isim))    
    
    "shp to Raster Tamam"
    ###########  Align Fc and Wc-Hc with Hc Image  ############
    fc  = rasterio.open(kayitYeri+'/'+"%s_Fc.tif" %(isim), mode="r").read(1)


    kwargs = hc.meta
    kwargs.update(dtype=rasterio.float32, count=1)

    with rasterio.open(kayitYeri+'/'+"%s_Fc.tif" %(isim), 'w', **kwargs) as dst:
        dst.write_band(1, fc.astype(rasterio.float32))
    print("Fc Oluşturuldu")




    wc  = rasterio.open(kayitYeri+'/'+"%s_Wc.tif" %(isim), mode="r").read(1)

    kwargs = hc.meta
    kwargs.update(dtype=rasterio.float32, count=1)

    with rasterio.open(kayitYeri+'/'+"%s_Wc.tif" %(isim), 'w', **kwargs) as dst:
        dst.write_band(1, wc.astype(rasterio.float32))
    print("wc Oluşturuldu")


    wc  = rasterio.open(kayitYeri+'/'+"%s_Wc.tif" %(isim), mode="r").read(1)
    wc_hc= wc/hc.read(1)
    wc_hc= np.where(wc_hc>0, wc_hc, 0)


    kwargs = hc.meta
    kwargs.update(dtype=rasterio.float32, count=1)

    with rasterio.open(kayitYeri+'/'+"%s_Wc_Hc.tif" %(isim), 'w', **kwargs) as dst:
        dst.write_band(1, wc_hc.astype(rasterio.float32))
    print("wc_hc Oluşturuldu")
    
    with rasterio.open(kayitYeri+'/'+"%s_Hc.tif" %(isim), 'w', **kwargs) as dst:
        dst.write_band(1, hc.read(1).astype(rasterio.float32))
    print("wc_hc Oluşturuldu")
    
    
    os.remove(kayitYeri+'/'+"%s_Wc.tif" %(isim))  #Wc yi silmek için
    os.remove(kayitYeri+'/'+"%s_grid.cpg" %(isim))  
    os.remove(kayitYeri+'/'+"%s_grid.dbf" %(isim))  
    os.remove(kayitYeri+'/'+"%s_grid.prj" %(isim))  
    os.remove(kayitYeri+'/'+"%s_grid.shp" %(isim)) 
    os.remove(kayitYeri+'/'+"%s_grid.shx" %(isim))  
    ###########  DF to Shp File  ############


Shp file dosyası Okundu


  0%|                                                   | 0/36 [00:00<?, ?it/s]

20200526_LAI Hesaplandı
toprak bitki ayrıştırması tamamlandı


C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Zonal istatistik tamamlandı
Vej Pixel sayıları Toplandı


...

Fc Oluşturuldu
wc Oluşturuldu
wc_hc Oluşturuldu
wc_hc Oluşturuldu
20200923 Klasörü Oluşturuldu!
20200923_LAI Hesaplandı
toprak bitki ayrıştırması tamamlandı


C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Zonal istatistik tamamlandı
Vej Pixel sayıları Toplandı


C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:119: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:119: RuntimeWarning: invalid value encountered in true_divide
100%|█████████████████████████████████████| 36/36 [23:19:47<00:00, 2332.99s/it]

Fc Oluşturuldu
wc Oluşturuldu
wc_hc Oluşturuldu
wc_hc Oluşturuldu
